<a href="https://colab.research.google.com/github/adidam/rag-impl/blob/main/tokenization_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [8]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# **Tokenize text to sentences with an overlap starting with a small size and increate to max length**

In [9]:
# Load tokenizer
MODEL_NAME = "BAAI/LLM-Embedder"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [10]:
def chunk_text_small_to_big(text, start_tokens=100, max_tokens=512, step=100, overlap_tokens=50):
    sentences = sent_tokenize(text)  # Tokenize text into sentences
    chunks = []
    i = 0
    size = start_tokens  # Start with small chunk

    while i < len(sentences):
        current_chunk = []
        current_length = 0
        j = i  # Track sentence index

        while j < len(sentences) and current_length + len(tokenizer.tokenize(sentences[j])) <= size:
            current_chunk.append(sentences[j])
            current_length += len(tokenizer.tokenize(sentences[j]))
            j += 1  # Move to next sentence

        if current_chunk:
            chunks.append(" ".join(current_chunk))

        # Increase chunk size gradually
        size = min(size + step, max_tokens)

        # Slide forward with overlap (backtrack a few sentences)
        i += max(1, (size - overlap_tokens) // 10)  # Convert tokens to ~sentence count

    return chunks


In [30]:
# Example Usage
from datasets import load_dataset
data = load_dataset("rungalileo/ragbench", 'techqa', split="train")

rows = data.select(range(2))
row = rows[0]
docs = row['documents']
txt = docs[2]
print(txt)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
chunks = chunk_text_small_to_big(txt)
for i, chunk in enumerate(chunks):
  print(f"Chunk {i+1} ({len(tokenizer.tokenize(chunk))} tokens): {chunk}\n")


z/os  A FIX IS AVAILABLE
Obtain the fix for this APAR.


SUBSCRIBE
You can track all active APARs for this component.



APAR STATUS
 * CLOSED AS UNREPRODUCIBLE IN NEXT RELEASE.
    
   
   

ERROR DESCRIPTION
 *  Development Fixes
   
   
    
   
   

LOCAL FIX

PROBLEM SUMMARY
 *  ****************************************************************
   * USERS AFFECTED:                                              *
   * All users of the IBM z/OS Communications Server for z/OS     *
   * Version 2 Release 1                                          *
   * E2827/K                                                      *
   ****************************************************************
   * PROBLEM DESCRIPTION:                                         *
   * 13245 EZBSRUTL(HIP6210 13.058)+00EE4A S0C4/00000010          *
   * 14704 AB/S0C4 EZBSRWCX +1D1E in TCPIP                        *
   * 14540 Stopped PFID(6) killed workloads running on PFID(5)    *
   * 14544 Symmetric links does not sw